Roughly 200K syllabus video resources, but only 17K are in the logs
-> Syllabus will be really poor predictor.

- Only 98 446 of interactions are of videos in a syllabus

-> Few videos in the logs are in the syllabus defined by the courses

- Every video in the log is in some syllabus
- End of syllabus -> Recommend MostPop 
- Out of Syllabus -> Recommend MostPop
- Fill out predictions -> MostPop

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import typing as t

In [2]:
courses = pd.read_json("../MOOCCubeX/entities/course.json", lines=True)

In [3]:
exploded = courses.drop(columns=["name", "prerequisites", "about"]).explode("resource")
course_flat = pd.concat([exploded.reset_index()["id"], pd.DataFrame(exploded["resource"].values.tolist())], axis=1).set_index("id")

In [4]:
%%time
# Assumes order is as published -> Generated order as chapter is not stable
# Or maybe just sort on chapter and replace "0" with "0.0.1" or something
courses_videos = course_flat[course_flat["resource_id"].str.startswith("V_")]
#courses_numbered = course_flat.assign(resource_order=course_flat.groupby("id")[["chapter"]].transform(lambda col: range(len(col))))

CPU times: user 142 ms, sys: 0 ns, total: 142 ms
Wall time: 147 ms


In [5]:
# Convert numbers to correct versions
invalid_chapters = courses_videos.loc[:,"chapter"].str.isnumeric()
courses_videos.loc[invalid_chapters, "chapter"] = courses_videos.loc[invalid_chapters, "chapter"].transform(lambda num: f"{num}.0")
courses_videos

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,titles,resource_id,chapter
id,,,
C_584313,"[第一课 导论与三家分晋, 导论, 导论]",V_849,1.1.1
C_584313,"[第一课 导论与三家分晋, 智伯的覆亡, 智伯的覆亡]",V_850,1.2.1
C_584313,"[第一课 导论与三家分晋, 智伯悲剧的反思, 智伯的覆亡讨论]",V_851,1.3.1
C_584313,"[第二课 战国前期的政治, 魏文侯治国, 魏文侯治国]",V_857,2.1.1
C_584313,"[第二课 战国前期的政治, 吴起的悲剧, 吴起的悲剧]",V_859,2.2.1
...,...,...,...
C_2329163,"[第四章 红色之旅实践写作, 4.15 论文写作规范, 4.15 论文写作规范]",V_8630133,3.14
C_2329163,"[第四章 红色之旅实践写作, 4.16 求职简历, 4.16 求职简历]",V_8630134,3.15
C_2329163,"[第四章 红色之旅实践写作, 4.17 实践—演讲稿, 4.17 实践—演讲稿]",V_8630135,3.16


In [6]:
video_list_by_natural_order = courses_videos.assign(next_video_id=courses_videos.groupby("id")["resource_id"].shift(-1))
video_list_by_natural_order

,titles,resource_id,chapter,next_video_id
id,,,,
C_584313,"[第一课 导论与三家分晋, 导论, 导论]",V_849,1.1.1,V_850
C_584313,"[第一课 导论与三家分晋, 智伯的覆亡, 智伯的覆亡]",V_850,1.2.1,V_851
C_584313,"[第一课 导论与三家分晋, 智伯悲剧的反思, 智伯的覆亡讨论]",V_851,1.3.1,V_857
C_584313,"[第二课 战国前期的政治, 魏文侯治国, 魏文侯治国]",V_857,2.1.1,V_859
C_584313,"[第二课 战国前期的政治, 吴起的悲剧, 吴起的悲剧]",V_859,2.2.1,V_861
...,...,...,...,...
C_2329163,"[第四章 红色之旅实践写作, 4.15 论文写作规范, 4.15 论文写作规范]",V_8630133,3.14,V_8630134
C_2329163,"[第四章 红色之旅实践写作, 4.16 求职简历, 4.16 求职简历]",V_8630134,3.15,V_8630135
C_2329163,"[第四章 红色之旅实践写作, 4.17 实践—演讲稿, 4.17 实践—演讲稿]",V_8630135,3.16,V_8630136


In [7]:
# Checking uniqueness of video_id within course
vid_lists = courses_videos.groupby("id")["resource_id"].agg(list)

In [8]:
for i in range(len(vid_lists)):
    set_out = set(vid_lists.iloc[i])
    for j in range(i+1, len(vid_lists)):
        set_in = set(vid_lists.iloc[j])
        if len(set_out.intersection(set_in)) > 0:
            print("Overlap between rows {i} and {j}")

KeyboardInterrupt: 

In [9]:
import json
with open("embeddings_fix/video_id2int.json") as f:
    video_id2int = json.load(f)

In [10]:
video_id2ccid = pd.read_csv("../MOOCCubeX/relations/video_id-ccid.txt", sep="\t", names=["video_id", "ccid"], index_col="video_id")
video_id2ccid

,ccid
video_id,
V_234845,0000363DB5B6E0869C33DC5901307461
V_234876,0000363DB5B6E0869C33DC5901307461
V_234907,0000363DB5B6E0869C33DC5901307461
V_293392,0000363DB5B6E0869C33DC5901307461
V_293445,0000363DB5B6E0869C33DC5901307461
...,...
V_6123337,FFFF5932EB4EC1A79C33DC5901307461
V_6123546,FFFF5932EB4EC1A79C33DC5901307461
V_6420181,FFFFD001230958A39C33DC5901307461


### Lookup process
1. Lookup video id
    - If not in lookup table:
        - video_id2ccid (OPT: partitioned by field (merge)) 
        - Find first syllabus_video_id from ccid
            -> Use that to look up

**Doesn't work because even if reversed video_ids -> Not certain that they are not OOV as well**

In [11]:
int2video_id = {num: video_id for video_id, num in video_id2int.items()}

In [12]:
most_pop_video = most_pop.copy()
most_pop_video.loc[:, "item_id"] = most_pop_video["item_id"].map(int2video_id)
most_pop_video

NameError: name 'most_pop' is not defined

In [44]:
most_pop_video["view_count"].sum()

1448772

In [46]:
most_pop_video[most_pop_video["item_id"].isin(courses_videos["resource_id"])].sum()

item_id       V_6074344V_6392107V_6392106V_6074382V_4412435V...
view_count                                                98446
dtype: object

In [52]:
syllabus_vid2ccid=courses_videos.merge(video_id2ccid, left_on="resource_id", right_on="video_id", how="outer")#.set_index("resource_id")
syllabus_vid2ccid

,titles,resource_id,chapter,ccid
0,"[第一课 导论与三家分晋, 导论, 导论]",V_849,1.1.1,ED9C27449691B2E29C33DC5901307461
1,"[第一课 导论与三家分晋, 智伯的覆亡, 智伯的覆亡]",V_850,1.2.1,E8B8262C55C901979C33DC5901307461
2,"[第一课 导论与三家分晋, 智伯悲剧的反思, 智伯的覆亡讨论]",V_851,1.3.1,133CE70E0CC0D11E9C33DC5901307461
3,"[第二课 战国前期的政治, 魏文侯治国, 魏文侯治国]",V_857,2.1.1,0C0D94E7D81D43199C33DC5901307461
4,"[第二课 战国前期的政治, 吴起的悲剧, 吴起的悲剧]",V_859,2.2.1,7A22C8182D07E81E9C33DC5901307461
...,...,...,...,...
2801284,NaN,NaN,NaN,FFFF3CC4C6F66F849C33DC5901307461
2801285,NaN,NaN,NaN,FFFF5932EB4EC1A79C33DC5901307461
2801286,NaN,NaN,NaN,FFFF5932EB4EC1A79C33DC5901307461
2801287,NaN,NaN,NaN,FFFFD001230958A39C33DC5901307461


In [ ]:
len(set(video_id2ccid.index.unique()).intersection(set(.keys())))

In [32]:
len(set(video_id2ccid.index.unique()).intersection(set(courses_videos["resource_id"].unique())))

201321

In [21]:
set(video_id2ccid.index.unique()).intersection(set(video_id2ccid.keys()))

set()

In [13]:
courses_videos["resource_id"].nunique()

203718

In [23]:
(video_list_by_natural_order["resource_id"].isin(video_id2int.keys())).sum()

17433

In [24]:
len(video_id2int.keys())

158062

In [25]:
len(set(video_id2int.keys()).intersection(set(video_list_by_natural_order["resource_id"].unique())))

17433

In [13]:
if (most_pop_path := Path("mooc/most_pop.parquet")).exists():
    most_pop = pd.read_parquet(most_pop_path)
else:
    interactions = pd.read_parquet("mooc/conventional/all_scaled/test_full.parquet", columns=["item_id"])    
    most_pop = interactions.value_counts().rename("view_count").reset_index()
    most_pop.to_parquet("mooc/most_pop.parquet")

#### Generate syllabus predictions

In [14]:
def listify(col, most_pop=most_pop["item_id"], top_k=20):
    next_cols = []    
    for idx, row in enumerate(col.iteritems()):
        # if X is nan (interacted item) -> Rec mostpop because can't reverse it
        if np.isnan(col.iloc[idx]):
            next_cols.append(most_pop[:top_k].tolist())
        else:
            # Fill in OOV items in the syllabus because low overlap between items and syllabus resources
            next_in_syllabus = col[idx+1:]
            is_nans = next_in_syllabus.isna()
            num_nans = is_nans.sum()
            nan_fill = pd.Series(most_pop[:num_nans].values, index=next_in_syllabus[is_nans].index, dtype=int)
            #print("num nans", num_nans)
            #print(nan_fill)
            #print("filled")
            #print(next_in_syllabus.fillna(nan_fill))
            next_cols.append((next_in_syllabus.fillna(nan_fill).astype(int).tolist() + most_pop[num_nans:top_k].tolist())[:top_k])
    return next_cols
#test = pd.DataFrame({"id":range(10)}, index=list("aabbccddee"))
#test.groupby(test.index)["id"].transform(listify)#.explode()


In [15]:
mapped_courses = courses_videos.copy()
mapped_courses.loc[:, "resource_id"] = mapped_courses["resource_id"].map(video_id2int)
#mapped_courses[:10].groupby("id")["resource_id"].transform(listify)

In [16]:
%%time
syllabus_predictions = mapped_courses.reset_index().groupby("id", group_keys=False)["resource_id"].transform(listify)

CPU times: user 14.1 s, sys: 154 ms, total: 14.2 s
Wall time: 14.8 s


In [17]:
syllabus_predictions

0         [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
1         [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
2         [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
3         [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
4         [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
                                ...                        
203713    [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
203714    [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
203715    [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
203716    [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
203717    [102, 103, 104, 105, 106, 109, 107, 110, 108, ...
Name: resource_id, Length: 203718, dtype: object

In [18]:
mapped_courses_filled = mapped_courses.reset_index().copy()
mapped_courses_filled.loc[:, "resource_id"] = mapped_courses_filled["resource_id"].fillna(value=-1)

In [19]:
syllabus_mapping = pd.concat([mapped_courses_filled["resource_id"].rename("item_id"), syllabus_predictions.rename("predictions")], axis=1)

In [20]:
syllabus_mapping # Can basically de-dupliacte if nan, nor potentially not really
# CAn't differentiate because don't know the video id since it isn't in the lookup so

,item_id,predictions
0,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
1,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
2,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
3,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
4,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
...,...,...
203713,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
203714,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
203715,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
203716,-1.0,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."


In [21]:
flat_predictions = pd.DataFrame(np.array(syllabus_mapping[["predictions"]].values.tolist()).squeeze())
flat_predictions

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
1,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
2,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
3,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
4,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203713,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
203714,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
203715,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160
203716,102,103,104,105,106,109,107,110,108,94,112,248,111,95,113,249,561,161,123,160


In [22]:
deduped_syllabus_mapping = syllabus_mapping.loc[flat_predictions.drop_duplicates().index].set_index("item_id")
deduped_syllabus_mapping.index = deduped_syllabus_mapping.index.astype(int)
deduped_syllabus_mapping

,predictions
item_id,
-1,"[102, 103, 104, 105, 106, 109, 107, 110, 108, ..."
28231,"[102, 103, 28232, 104, 35700, 35701, 105, 2057..."
28232,"[102, 35700, 35701, 103, 20579, 104, 105, 5274..."
35700,"[35701, 102, 20579, 103, 104, 52747, 105, 5031..."
35701,"[102, 20579, 103, 104, 52747, 105, 50310, 1479..."
...,...
3545,"[122109, 35241, 34913, 34914, 49605, 102, 103,..."
122109,"[35241, 34913, 34914, 49605, 102, 103, 104, 10..."
35241,"[34913, 34914, 49605, 102, 103, 104, 105, 106,..."


In [23]:
deduped_syllabus_mapping.to_parquet("mooc/deduped_syllabus_mapping.parquet")